# 아파트 실거래가 예측

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import  LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from gensim.models.word2vec import Word2Vec
import xgboost as xgb
import lightgbm as lgb
import re
import optuna

sns.set_theme(style="darkgrid")


## Data Load

In [2]:
train_df = pd.read_csv('./input/train.csv')
test_df = pd.read_csv('./input/test.csv')

### apart

In [3]:
#괄호와 괄호안내용 제거
regex = "\(.*\)|\s-\s.*"
for i in tqdm(range(len(train_df))):
   train_df.at[i, 'apt'] = re.sub(regex, '', train_df.at[i, 'apt'])
for i in tqdm(range(len(test_df))):
   test_df.at[i, 'apt'] = re.sub(regex, '', test_df.at[i, 'apt'])   

100%|██████████| 5463/5463 [00:00<00:00, 121724.19it/s]


In [4]:
# top 10 시공사 아파트 여부를 나타내는 컬럼 생성
train_df['top10'] = 0
test_df['top10'] = 0
top10 = ['자이', '푸르지오', '더샵', '롯데캐슬', '이편한|e편한|e-편한',
         '힐스테이트', '아이파크', '래미안', 'sk|SK|에스케이', '데시앙']

train_df['apt'] = train_df['apt'].fillna('others')
# top 10 시공사면 1, 아니면 0
for i, brand in enumerate(top10):
    train_df.loc[train_df['apt'].str.contains(brand), 'top10'] = 1
    test_df.loc[test_df['apt'].str.contains(brand), 'top10'] = 1


In [5]:
# model = Word2Vec([train_df['apt'].values.tolist()], min_count=1, vector_size=1024, sg=1, epochs=100)
# model.save('apt_w2v_1024_sg_100.model')

In [5]:
apt_w2v = Word2Vec.load('apt_w2v_1024_sg_100.model')

In [6]:
apt_w2v.wv.most_similar('현대')

[('삼성', 0.9643094539642334),
 ('구암하이빌', 0.9511270523071289),
 ('세종아파트', 0.9471513032913208),
 ('프라임1', 0.9401869177818298),
 ('그린빌', 0.9393579363822937),
 ('청호그린빌', 0.9378136396408081),
 ('프라임캐슬', 0.9373813271522522),
 ('무악다온채', 0.936416506767273),
 ('동산빌라', 0.9331886172294617),
 ('인왕산2차아이파크', 0.9328047037124634)]

In [7]:
train_df['apt_embedded'] = train_df['apt'].apply(lambda x: apt_w2v.wv[x])

In [9]:
train_df.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,top10,apt_embedded
0,0,7622,서울특별시,신교동,6-13,신현,신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500,0,"[-0.10752576, -0.18122844, -0.07227908, 0.0448..."
1,1,5399,서울특별시,필운동,142,사직파크맨션,필운동 142 사직파크맨션,99.17,1973,200801,1~10,6,20000,0,"[-0.053165514, -0.1006416, -0.07624212, 0.0012..."
2,2,3578,서울특별시,필운동,174-1,두레엘리시안,필운동 174-1 두레엘리시안,84.74,2007,200801,1~10,6,38500,0,"[-0.06343027, -0.19215204, -0.06197246, 0.0544..."
3,3,10957,서울특별시,내수동,95,파크팰리스,내수동 95 파크팰리스,146.39,2003,200801,11~20,15,118000,0,"[-0.036577757, -0.24322, -0.1593426, 0.0263538..."
4,4,10639,서울특별시,내수동,110-15,킹스매너,내수동 110-15 킹스매너,194.43,2004,200801,21~31,3,120000,0,"[-0.02274445, -0.20998694, -0.12684277, 0.0276..."


### date
apt 전처리는 마쳤으니, 이번에는 date로 넘어가봅시다.

In [8]:
# test 시작 거래연월인 인덱스 저장
test_start = train_df.loc[train_df['transaction_year_month'] == 201701, 'transaction_year_month'].index[0]

In [9]:
# 완공연도에서 최소연도를 뺌으로써 완공연도 라벨인코딩
print('변환전\n', train_df['year_of_completion'].unique()[:5])
train_df['year_of_completion'] = train_df['year_of_completion'] - train_df['year_of_completion'].min()
test_df['year_of_completion'] = test_df['year_of_completion'] - test_df['year_of_completion'].min()
print('변환후\n', train_df['year_of_completion'].unique()[:5])

# 연월 증가하는 순으로 라벨 인코딩
print('train 변환전\n', train_df['transaction_year_month'].unique()[:5])
print('test 변환전\n', test_df['transaction_year_month'].unique()[:5])
le = LabelEncoder()
train_df['transaction_year_month'] = le.fit_transform(train_df['transaction_year_month'])
# test는 다음과 같이 처리
test_df['transaction_year_month'] = test_df['transaction_year_month'] - test_df['transaction_year_month'].min() + train_df.at[test_start, 'transaction_year_month']
print('train 변환후\n', train_df['transaction_year_month'].unique()[:5])
print('test 변환후\n', test_df['transaction_year_month'].unique()[:5])

# 필요없는 열 제거
train_df = train_df.drop(['jibun', 'transaction_date', 'addr_kr'], axis=1)
test_df = test_df.drop(['jibun', 'transaction_date', 'addr_kr'], axis=1)

변환전
 [2002 1973 2007 2003 2004]
변환후
 [41 12 46 42 43]
train 변환전
 [200801 200802 200803 200804 200805]
test 변환전
 [201711 201708 201710 201707 201712]
train 변환후
 [0 1 2 3 4]
test 변환후
 [118 115 117 114 119]


### dong
- 이번에는 주소의 동입니다.
- 먼저 서울과 부산에서 같은 이름을 가진 동이 있는지 확인해보겠습니다.

In [10]:
seoul_set = set(train_df.loc[train_df['city']=='서울특별시', 'dong'])
busan_set = set(train_df.loc[train_df['city']=='부산광역시', 'dong'])
same_dong = seoul_set & busan_set 
print(same_dong)

{'중동', '사직동', '부암동', '송정동'}


- 서울과 부산에 중동, 부암동, 송정동, 사직동 총 네 동이 겹칩니다.
- 접두사에 서울 또는 부산을 붙여 같은동을 분리하겠습니다.

In [11]:
for d in same_dong:
    train_df.loc[(train_df['city']=='서울특별시') & (train_df['dong']==d), 'dong'] = '서울' + d
    train_df.loc[(train_df['city']=='부산광역시') & (train_df['dong']==d), 'dong'] = '부산' + d
    test_df.loc[(test_df['city']=='서울특별시') & (test_df['dong']==d), 'dong'] = '서울' + d
    test_df.loc[(test_df['city']=='부산광역시') & (test_df['dong']==d), 'dong'] = '부산' + d
    

seoul_set = set(train_df.loc[train_df['city']=='서울특별시', 'dong'])
busan_set = set(train_df.loc[train_df['city']=='부산광역시', 'dong'])
same_dong = seoul_set & busan_set
print(same_dong)  

set()


In [12]:
# model = Word2Vec([train_df['dong'].values.tolist()], min_count=1, vector_size=1024, sg=1, epochs=100)
# model.save('dong_w2v_1024_sg_100.model')
dong_w2v = Word2Vec.load('dong_w2v_1024_sg_100.model')
train_df['dong_embedded'] = train_df['dong'].apply(lambda x: dong_w2v.wv[x])
train_df.head()

,transaction_id,apartment_id,city,dong,apt,exclusive_use_area,year_of_completion,transaction_year_month,floor,transaction_real_price,top10,apt_embedded,dong_embedded
0,0,7622,서울특별시,신교동,신현,84.82,41,0,2,37500,0,"[-0.10752576, -0.18122844, -0.07227908, 0.0448...","[0.17256321, -0.0375122, 0.09505555, 0.0194796..."
1,1,5399,서울특별시,필운동,사직파크맨션,99.17,12,0,6,20000,0,"[-0.053165514, -0.1006416, -0.07624212, 0.0012...","[0.22832519, 0.015710473, 0.1406205, -0.012602..."
2,2,3578,서울특별시,필운동,두레엘리시안,84.74,46,0,6,38500,0,"[-0.06343027, -0.19215204, -0.06197246, 0.0544...","[0.22832519, 0.015710473, 0.1406205, -0.012602..."
3,3,10957,서울특별시,내수동,파크팰리스,146.39,42,0,15,118000,0,"[-0.036577757, -0.24322, -0.1593426, 0.0263538...","[0.12117733, -0.03002017, 0.12184901, -0.05570..."
4,4,10639,서울특별시,내수동,킹스매너,194.43,43,0,3,120000,0,"[-0.02274445, -0.20998694, -0.12684277, 0.0276...","[0.12117733, -0.03002017, 0.12184901, -0.05570..."


### Floor

In [15]:
train_df.describe()

,transaction_id,apartment_id,exclusive_use_area,year_of_completion,transaction_year_month,floor,transaction_real_price,top10
count,1.216553e+06,1.216553e+06,1.216553e+06,1.216553e+06,1.216553e+06,1.216553e+06,1.216553e+06,1.216553e+06
mean,6.091530e+05,6.299685e+03,7.816549e+01,3.729657e+01,6.437834e+01,9.343291e+00,3.822769e+04,1.234225e-01
std,3.526198e+05,3.581169e+03,2.915113e+01,8.941347e+00,3.509363e+01,6.606500e+00,3.104898e+04,3.289217e-01
min,0.000000e+00,0.000000e+00,9.260000e+00,0.000000e+00,0.000000e+00,-4.000000e+00,1.000000e+02,0.000000e+00
25%,3.041380e+05,3.345000e+03,5.976000e+01,3.200000e+01,3.300000e+01,4.000000e+00,1.900000e+04,0.000000e+00
50%,6.082760e+05,5.964000e+03,8.241000e+01,3.800000e+01,7.100000e+01,8.000000e+00,3.090000e+04,0.000000e+00
75%,9.124140e+05,9.436000e+03,8.497000e+01,4.400000e+01,9.400000e+01,1.300000e+01,4.700000e+04,0.000000e+00
max,1.234827e+06,1.265800e+04,4.243200e+02,5.600000e+01,1.180000e+02,8.000000e+01,8.200000e+05,1.000000e+00


- 최소층이 -4층임을 확인할 수 있습니다.
- 4를 더해서 라벨인코딩을 진행해줍니다.
- test는 최소층이 -1층이었으니 맞게 변환합니다.

In [13]:
# 최소값이 -4이므로 4를 더해서 음수를 없애고 순서형범주처리
print('변환전\n', train_df['floor'].values[:5])
train_df['floor'] = train_df['floor'].map(lambda x: x+4)
test_df['floor'] = test_df['floor'].map(lambda x: x+1)
print('변환후\n', train_df['floor'].values[:5])

변환전
 [ 2  6  6 15  3]
변환후
 [ 6 10 10 19  7]


### Price
- 이제 타겟데이터를 살펴보겠습니다.

- 가격의 분포가 매우 왼쪽으로 치우친 것을 확인할 수 있습니다.
- 타겟 변수의 이상치가 회귀모형을 사용한 예측에 큰 영향을 줄 수 있으니,
    - 로그 변환으로 정규화한뒤, 나중에 다시 역변환하겠습니다!

In [14]:
# 가격 로그 변환 후 원래 가격 따로 저장
train_df['log_price'] = np.log1p(train_df['transaction_real_price'])
real_price = train_df['transaction_real_price'] # 원래 가격
train_df.drop('transaction_real_price', axis=1, inplace=True)
train_df.head(1)

,transaction_id,apartment_id,city,dong,apt,exclusive_use_area,year_of_completion,transaction_year_month,floor,top10,apt_embedded,dong_embedded,log_price
0,0,7622,서울특별시,신교동,신현,84.82,41,0,6,0,"[-0.10752576, -0.18122844, -0.07227908, 0.0448...","[0.17256321, -0.0375122, 0.09505555, 0.0194796...",10.532123


In [15]:
# 면적 로그 변환 후 원래 면적 따로 저장
train_df['log_area'] = np.log1p(train_df['exclusive_use_area'])
test_df['log_area'] = np.log1p(test_df['exclusive_use_area'])
area = train_df['exclusive_use_area'] # 원래 가격
train_df.drop('exclusive_use_area', axis=1, inplace=True)
test_df.drop('exclusive_use_area', axis=1, inplace=True)
train_df.head(1)

,transaction_id,apartment_id,city,dong,apt,year_of_completion,transaction_year_month,floor,top10,apt_embedded,dong_embedded,log_price,log_area
0,0,7622,서울특별시,신교동,신현,41,0,6,0,"[-0.10752576, -0.18122844, -0.07227908, 0.0448...","[0.17256321, -0.0375122, 0.09505555, 0.0194796...",10.532123,4.452252


### Encoding
- 인코딩은 이제 `city`만 진행하면 됩니다.
- `city`는 서울특별시면 1, 부산광역시면 0으로 변환하겠습니다.
- 학습에 사용하지 않을 피쳐들도 제거하겠습니다.

In [16]:
drop_col = ['transaction_id', 'apartment_id', 'dong', 'apt']

train_df['city'] = train_df['city'].map(lambda x: 1 if x == '서울특별시' else 0)
test_df['city'] = test_df['city'].map(lambda x: 1 if x == '서울특별시' else 0)

In [17]:
train_df.drop(drop_col, axis=1, inplace=True)
test_df.drop(drop_col, axis=1, inplace=True)
train_df.head(1)

,city,year_of_completion,transaction_year_month,floor,top10,apt_embedded,dong_embedded,log_price,log_area
0,1,41,0,6,0,"[-0.10752576, -0.18122844, -0.07227908, 0.0448...","[0.17256321, -0.0375122, 0.09505555, 0.0194796...",10.532123,4.452252


In [21]:
train_df.head()

,city,year_of_completion,transaction_year_month,floor,top10,apt_embedded,dong_embedded,log_price,log_area
0,1,41,0,6,0,"[-0.10752576, -0.18122844, -0.07227908, 0.0448...","[0.17256321, -0.0375122, 0.09505555, 0.0194796...",10.532123,4.452252
1,1,12,0,10,0,"[-0.053165514, -0.1006416, -0.07624212, 0.0012...","[0.22832519, 0.015710473, 0.1406205, -0.012602...",9.903538,4.606869
2,1,46,0,10,0,"[-0.06343027, -0.19215204, -0.06197246, 0.0544...","[0.22832519, 0.015710473, 0.1406205, -0.012602...",10.558439,4.451319
3,1,42,0,19,0,"[-0.036577757, -0.24322, -0.1593426, 0.0263538...","[0.12117733, -0.03002017, 0.12184901, -0.05570...",11.678448,4.993082
4,1,43,0,7,0,"[-0.02274445, -0.20998694, -0.12684277, 0.0276...","[0.12117733, -0.03002017, 0.12184901, -0.05570...",11.695255,5.275202


In [18]:
emb_apt = np.array(train_df['apt_embedded'].to_numpy().tolist())
emb_dong = np.array(train_df['dong_embedded'].to_numpy().tolist())
print(emb_apt[0])
print(emb_dong[0])
print(emb_apt.shape, emb_dong.shape)

[-0.10752576 -0.18122844 -0.07227908 ... -0.21428467 -0.07581832
 -0.16598397]
[ 0.17256321 -0.0375122   0.09505555 ... -0.1406364  -0.08027103
  0.09223551]
(1216553, 1024) (1216553, 1024)


In [19]:
X_common = train_df.drop(['apt_embedded', 'dong_embedded'], axis=1)
embedding = np.concatenate([emb_apt, emb_dong], axis=1)
train_X = np.concatenate([X_common.drop('log_price', axis=1).to_numpy(), embedding], axis=1)
train_y = train_df['log_price'].to_numpy()

In [24]:
print(len(X_common.columns), len(embedding[0]))

7 2048


In [26]:
print(train_X.shape, train_y.shape)

(1216553, 2054) (1216553,)


In [ ]:
# import tensorflow as tf
# train_df['apt_embedded'] = train_df['apt_embedded'].apply(lambda x: tf.reshape(x, [-1]).numpy())
# train_df['dong_embedded'] = train_df['dong_embedded'].apply(lambda x: tf.reshape(x, [-1]).numpy())
# train_X, train_y = train_df.drop(['log_price'], axis=1), train_df['log_price']
# print(train_X.shape, train_y.shape)
# train_df['apt_embedded'] = np.array(train_df['apt_embedded'].to_numpy().tolist())
# train_df['dong_embedded'] = np.array(train_df['dong_embedded'].to_numpy().tolist())
# train_df['apt_embedded']

In [20]:
def RMSE(y, y_pred):
    rmse = mean_squared_error(y, y_pred) ** 0.5
    return rmse

- 간단하게 모델을 정의합니다.

In [21]:
cut = int(len(train_df)*0.8)
h_train_X = train_X[:cut]
h_train_y = train_y[:cut]
h_valid_X = train_X[cut:]
h_valid_y = train_y[cut:]

# h_train = train_X[:cut]
# h_valid = train_df[cut:]

# h_train_X = h_train.drop('log_price', axis=1)
# h_train_y = h_train['log_price']
# h_valid_X = h_valid.drop('log_price', axis=1)
# h_valid_y = h_valid['log_price']
# # dataframe to numpy array
# h_train_X, h_train_y, h_valid_X, h_valid_y = h_train_X.to_numpy(), h_train_y.to_numpy(), h_valid_X.to_numpy(), h_valid_y.to_numpy()
print(h_train_X.shape, h_train_y.shape, h_valid_X.shape, h_valid_y.shape)

(973242, 2054) (973242,) (243311, 2054) (243311,)


In [29]:
h_train_X[0]

array([ 1.        , 41.        ,  0.        , ..., -0.1406364 ,
       -0.08027103,  0.09223551])

In [30]:
h_valid_X[0]

array([ 0.00000000e+00,  3.40000000e+01,  4.10000000e+01, ...,
       -4.04775841e-04,  8.62445915e-04, -3.07057984e-04])

In [31]:
h_valid_y[0]

9.680406499268875

In [23]:
from optuna.samplers import TPESampler

sampler = TPESampler(seed=10)

def objective(trial):
    dtrain = lgb.Dataset(h_train_X, label=h_train_y)
    dtest = lgb.Dataset(h_valid_X, label=h_valid_y)

    param = {
        'objective': 'regression', # 회귀
        'verbose': -1,
        'device': 'gpu',
        'metric': 'rmse', 
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }

    model = lgb.LGBMRegressor(**param)
    lgb_model = model.fit(h_train_X, h_train_y, eval_set=[(h_valid_X, h_valid_y)], verbose=0, early_stopping_rounds=25)
    rmse = RMSE(h_valid_y, lgb_model.predict(h_valid_X))
    return rmse
        
study_lgb = optuna.create_study(direction='minimize', sampler=sampler)
study_lgb.optimize(objective, n_trials=100)

[I 2021-10-08 14:41:03,291] A new study created in memory with name: no-name-47321b27-c1f8-46c1-bef4-6b52e2c9d64e
/home/kok/anaconda3/envs/rjs/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kok/anaconda3/envs/rjs/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


KeyboardInterrupt: 

In [ ]:
trial = study_lgb.best_trial
trial_params = trial.params
print('Best Trial: score {},\nparams {}'.format(trial.value, trial_params))

- 처음에 LightGBM 평균 RMSLE가 0.2390임을 감안했을때, 매우 성능이 향상됐음을 알 수 있습니다.

## LightGBM Submission
- 이제 test 셋을 사용해서 inference를 해보고 실제 점수를 Dacon에서 확인해볼시간입니다.
- 먼저 train 데이터에서 진행한 모든 전처리를 test 데이터에도 적용합니다.

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
final_lgb_model = lgb.LGBMRegressor(**trial_params)
final_lgb_model.fit(train_X, train_y)
final_lgb_pred = final_lgb_model.predict(test_df)

In [ ]:
final_lgb_pred

In [ ]:
plt.barh(train_X.columns, final_lgb_model.feature_importances_)

- 동, 완공연도, 면적, 아파트명 순으로 중요도가 높다고 나옵니다.
- top 시공사는 별로 중요하지 않다고 하네요..

## Submission
- 이제 제출을 해봅시다.
- 가격을 다시 역변환합니다.

In [ ]:
final_pred_sub = np.expm1(final_lgb_pred)
final_pred_sub

In [ ]:
sub = pd.read_csv('./input/test.csv')
sub_df = pd.DataFrame({'transaction_id': sub['transaction_id'], 'transaction_real_price': final_pred_sub})
sub_df

In [ ]:
sub_df.to_csv('submission_lgb.csv', index=False)

여기까지 따라오시느라 고생하셨습니다.

긴 글 봐주셔서 감사합니다:)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import  LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
import re
import optuna

sns.set_theme(style="darkgrid")


## Data Load

In [ ]:
train_df = pd.read_csv('./input/train.csv')
test_df = pd.read_csv('./input/test.csv')

In [ ]:
train_df.head(3)

In [ ]:
test_df.head(3)

- exclusive_use_area: 전용면적(한 세대만 독점적으로 사용하는 공간)
- transaction_real_price: 실거래가(단위:만원, 타겟 값, train만 존재)
- transaction_year_month: 거래년월
- transaction_date: 거래일
- floor: 층 

- 데이터가 거래시간 순으로 이루어져 있습니다.

In [ ]:
print(train_df.shape)
print(test_df.shape)

- 처음부터 train 데이터와 test데이터가 분리되어 있습니다. (id가 겹치지 않습니다.)
    - train data: 1216552행 13열로 구성
    - test data: 5463행 12열로 구성 (예측변수 'transaction_real_price'는 제외됨)

In [ ]:
train_df.head(5)

In [ ]:
train_df.describe() 

In [ ]:
train_df.info()

- train_df 결측치는 없습니다.
- 건물들은 1961년 ~ 2017년에 완공됐습니다.
- 9.26m^2 ~ 424.32m^2 전용면적을 가지고 있습니다.
- Floor의 경우 음의 값이 존재합니다.
- 실거래가는 100만원부터 820억까지 존재합니다.

In [ ]:
train_df.loc[train_df['transaction_real_price']==820000]

In [ ]:
test_df.describe()

- test 데이터는 거래 연월이 2017년 데이터만 존재합니다.
- 완공연도는 train과 동일합니다.
- 층은 -1층부터 존재합니다.

In [ ]:
test_df.info()

- 마찬가지로 결측치는 없습니다.

## Preprocessing

### apart

In [ ]:
rain_df.head(1)

- 신현(101동)과 신현(102동)의 가격차이가 클까요?
- apt 이름에 존재하는 괄호 + 괄호안 내용을 제거해서 통일하겠습니다.

In [ ]:
#괄호와 괄호안내용 제거
regex = "\(.*\)|\s-\s.*"
for i in tqdm(range(len(train_df))):
   train_df.at[i, 'apt'] = re.sub(regex, '', train_df.at[i, 'apt'])
for i in tqdm(range(len(test_df))):
   test_df.at[i, 'apt'] = re.sub(regex, '', test_df.at[i, 'apt'])   

In [ ]:
train_df['apt'].value_counts()[:20]

In [ ]:
train_df['apt'].nunique()

In [ ]:
# top 10 시공사 아파트 여부를 나타내는 컬럼 생성
train_df['top10'] = 0
test_df['top10'] = 0
top10 = ['자이', '푸르지오', '더샵', '롯데캐슬', '이편한|e편한|e-편한',
         '힐스테이트', '아이파크', '래미안', 'sk|SK|에스케이', '데시앙']

train_df['apt'] = train_df['apt'].fillna('others')
# top 10 시공사면 1, 아니면 0
for i, brand in enumerate(top10):
    train_df.loc[train_df['apt'].str.contains(brand), 'top10'] = 1
    test_df.loc[test_df['apt'].str.contains(brand), 'top10'] = 1


In [ ]:
train_df.head(1)

In [ ]:
from gensim.models.word2vec import Word2Vec
model = Word2Vec([train_df['apt'].values.tolist()], min_count=1, vector_size=1024, sg=1, epochs=100)
model.save('apt_w2v_1024_sg_100.model')

In [ ]:
apt_w2v = Word2Vec.load('apt_w2v_1024_sg_100.model')

In [ ]:
apt_w2v.wv.most_similar('현대')

In [ ]:
dong_w2v.wv.most_similar('신교동')[-1]

In [ ]:
train_df['apt_embedded'] = train_df['apt'].apply(lambda x: apt_w2v.wv[x])

In [ ]:
train_df.head()

### date
apt 전처리는 마쳤으니, 이번에는 date로 넘어가봅시다.

In [ ]:
train_df.head(1)

In [ ]:
train_df.describe()

- 날짜 관련 컬럼은 `year_of_completion`, `transaction_year_month`, `transaction_date`가 있습니다.
- `transaction_date`는 0~10, 11~20, 21~30 총 세가지로 이루어져 있는데, 가격에 크게 영향을 미칠 것 같지 않아 제외합니다.
각 컬럼의 최대/최소가
- `year_of_completion`
    - 최소연도: 1961년
    - 최대연도: 2017년
- `transaction_year_month`
    - 최소연도: 2008년 1월
    - 최대연도: 2017년 11월
임을 확인합니다.
- 최대연도에서 최소연도를 빼면 정수형 라벨인코딩이 완성되겠죠?
- 사용하지 않을 열도 미리 제거하겠습니다.

주의할 점은 test 데이터의 라벨인코딩입니다.
- test 완공연도는 train 데이터와 동일하기 때문에 상관없습니다.
- 대신 거래연월이 2017년 01월부터 12월까지로만 이루어져 있기 때문에, 최소값을 빼면 train 라벨인코딩과 다른 값이 됩니다.
- 따라서 test의 거래연월에서 2017을 뺀 값에서 201701의 인코딩 값을 더해줍니다.


In [ ]:
# test 시작 거래연월인 인덱스 저장
test_start = train_df.loc[train_df['transaction_year_month'] == 201701, 'transaction_year_month'].index[0]

In [ ]:
# 완공연도에서 최소연도를 뺌으로써 완공연도 라벨인코딩
print('변환전\n', train_df['year_of_completion'].unique()[:5])
train_df['year_of_completion'] = train_df['year_of_completion'] - train_df['year_of_completion'].min()
test_df['year_of_completion'] = test_df['year_of_completion'] - test_df['year_of_completion'].min()
print('변환후\n', train_df['year_of_completion'].unique()[:5])

# 연월 증가하는 순으로 라벨 인코딩
print('train 변환전\n', train_df['transaction_year_month'].unique()[:5])
print('test 변환전\n', test_df['transaction_year_month'].unique()[:5])
le = LabelEncoder()
train_df['transaction_year_month'] = le.fit_transform(train_df['transaction_year_month'])
# test는 다음과 같이 처리
test_df['transaction_year_month'] = test_df['transaction_year_month'] - test_df['transaction_year_month'].min() + train_df.at[test_start, 'transaction_year_month']
print('train 변환후\n', train_df['transaction_year_month'].unique()[:5])
print('test 변환후\n', test_df['transaction_year_month'].unique()[:5])

# 필요없는 열 제거
train_df = train_df.drop(['jibun', 'transaction_date', 'addr_kr'], axis=1)
test_df = test_df.drop(['jibun', 'transaction_date', 'addr_kr'], axis=1)

In [ ]:
train_df.head(5)

In [ ]:
test_df

In [ ]:
train_df.describe()

In [ ]:
test_df.describe()

### dong
- 이번에는 주소의 동입니다.
- 먼저 서울과 부산에서 같은 이름을 가진 동이 있는지 확인해보겠습니다.

In [ ]:
seoul_set = set(train_df.loc[train_df['city']=='서울특별시', 'dong'])
busan_set = set(train_df.loc[train_df['city']=='부산광역시', 'dong'])
same_dong = seoul_set & busan_set 
print(same_dong)

seoul_set = set(test_df.loc[test_df['city']=='서울특별시', 'dong'])
busan_set = set(test_df.loc[test_df['city']=='부산광역시', 'dong'])
same_dong = seoul_set & busan_set 
print(same_dong)

- 서울과 부산에 중동, 부암동, 송정동, 사직동 총 네 동이 겹칩니다.
- 접두사에 서울 또는 부산을 붙여 같은동을 분리하겠습니다.

In [ ]:
for d in same_dong:
    train_df.loc[(train_df['city']=='서울특별시') & (train_df['dong']==d), 'dong'] = '서울' + d
    train_df.loc[(train_df['city']=='부산광역시') & (train_df['dong']==d), 'dong'] = '부산' + d
    test_df.loc[(test_df['city']=='서울특별시') & (test_df['dong']==d), 'dong'] = '서울' + d
    test_df.loc[(test_df['city']=='부산광역시') & (test_df['dong']==d), 'dong'] = '부산' + d
    

seoul_set = set(train_df.loc[train_df['city']=='서울특별시', 'dong'])
busan_set = set(train_df.loc[train_df['city']=='부산광역시', 'dong'])
same_dong = seoul_set & busan_set
print(same_dong)  

- 더 이상 겹치는 동이 없습니다.
- 이번에는 동별로 평균 가격을 확인해보겠습니다.
- 아파트 평균 가격을 확인하는 방법과 동일합니다.

In [ ]:
model = Word2Vec([train_df['dong'].values.tolist()], min_count=1, vector_size=1024, sg=1, epochs=100)
model.save('dong_w2v_1024_sg_100.model')
dong_w2v = Word2Vec.load('dong_w2v_1024_sg_100.model')
train_df['dong_embedded'] = train_df['dong'].apply(lambda x: dong_w2v.wv[x])
train_df

### Floor

In [ ]:
train_df.describe()

- 최소층이 -4층임을 확인할 수 있습니다.
- 4를 더해서 라벨인코딩을 진행해줍니다.
- test는 최소층이 -1층이었으니 맞게 변환합니다.

In [ ]:
# 최소값이 -4이므로 4를 더해서 음수를 없애고 순서형범주처리
print('변환전\n', train_df['floor'].values[:5])
train_df['floor'] = train_df['floor'].map(lambda x: x+4)
test_df['floor'] = test_df['floor'].map(lambda x: x+1)
print('변환후\n', train_df['floor'].values[:5])

### Price
- 이제 타겟데이터를 살펴보겠습니다.

In [ ]:
# train price
plt.figure()
sns.displot(train_df['transaction_real_price'], bins=30)
plt.xlabel('Price(10000 won)')
plt.title('Distribution of Price')
plt.show()

- 가격의 분포가 매우 왼쪽으로 치우친 것을 확인할 수 있습니다.
- 타겟 변수의 이상치가 회귀모형을 사용한 예측에 큰 영향을 줄 수 있으니,
    - 로그 변환으로 정규화한뒤, 나중에 다시 역변환하겠습니다!

In [ ]:
# 가격 로그 변환 후 원래 가격 따로 저장
train_df['log_price'] = np.log1p(train_df['transaction_real_price'])
real_price = train_df['transaction_real_price'] # 원래 가격
train_df.drop('transaction_real_price', axis=1, inplace=True)
train_df.head(1)

- 잘 변환이 됐는지 그래프를 그려보겠습니다.

In [ ]:
f, (ax1, ax2) = plt.subplots(1,2,figsize=(12,6))

ax1.hist(real_price, bins=30)
ax1.set_title('Price Distribution')
ax1.set_xlabel('Price')

ax2.hist(train_df['log_price'], bins=30)
ax2.set_title('Log Price Distribution')
ax2.set_xlabel('Log Price')

plt.show()

- 가격이 정규분포처럼 잘 근사됐네요 :)

### Area
- 면적도 가격과 동일하게!

In [ ]:
# train area
plt.figure()
sns.displot(train_df['exclusive_use_area'], bins=30)
plt.xlabel('Area(Square meter)')
plt.title('Distribution of Area')
plt.show()

In [ ]:
# 면적 로그 변환 후 원래 면적 따로 저장
train_df['log_area'] = np.log1p(train_df['exclusive_use_area'])
test_df['log_area'] = np.log1p(test_df['exclusive_use_area'])
area = train_df['exclusive_use_area'] # 원래 가격
train_df.drop('exclusive_use_area', axis=1, inplace=True)
test_df.drop('exclusive_use_area', axis=1, inplace=True)
train_df.head(1)

In [ ]:
f, (ax1, ax2) = plt.subplots(1,2,figsize=(12,6))

ax1.hist(area, bins=30)
ax1.set_title('Distribution of Area')
ax1.set_xlabel('Area')

ax2.hist(train_df['log_area'], bins=30)
ax2.set_title('Distribution of Log Area')
ax2.set_xlabel('Area')

plt.show()

- 가격만큼 만족스럽진 않지만.. 그냥 쓰겠습니다 ㅎㅎ

### Encoding
- 인코딩은 이제 `city`만 진행하면 됩니다.
- `city`는 서울특별시면 1, 부산광역시면 0으로 변환하겠습니다.
- 학습에 사용하지 않을 피쳐들도 제거하겠습니다.

In [ ]:
drop_col = ['transaction_id', 'apartment_id', 'dong', 'apt']

train_df['city'] = train_df['city'].map(lambda x: 1 if x == '서울특별시' else 0)
test_df['city'] = test_df['city'].map(lambda x: 1 if x == '서울특별시' else 0)

In [ ]:
train_df.drop(drop_col, axis=1, inplace=True)
test_df.drop(drop_col, axis=1, inplace=True)
train_df.head(1)

In [ ]:
test_df.head(1)

In [ ]:
train_df.info()

In [ ]:
print(train_df.shape, test_df.shape)

In [116]:
import tensorflow as tf
train_df['apt_embedded'] = train_df['apt_embedded'].apply(lambda x: tf.reshape(x, [-1]).numpy())
train_df['dong_embedded'] = train_df['dong_embedded'].apply(lambda x: tf.reshape(x, [-1]).numpy())
train_X, train_y = train_df.drop(['log_price', 'dong', 'apt'], axis=1), train_df['log_price']
print(train_X.shape, train_y.shape)

In [ ]:
def RMSE(y, y_pred):
    rmse = mean_squared_error(y, y_pred) ** 0.5
    return rmse

- 간단하게 모델을 정의합니다.

In [ ]:
reg = LinearRegression(n_jobs=-1)
ridge = Ridge(alpha=0.8, random_state=1)
lasso = Lasso(alpha = 0.01, random_state=1)
Enet = ElasticNet(alpha=0.03, l1_ratio=0.01, random_state=1)
DTree = DecisionTreeRegressor(max_depth=6, min_samples_split=10, min_samples_leaf=15, random_state=1)
rf = RandomForestRegressor(n_estimators=500, criterion='mse', max_depth=9, min_samples_split=50,
                           min_samples_leaf=5, random_state=1, n_jobs=-1)
model_xgb = xgb.XGBRegressor(n_estimators=500, max_depth=9, min_child_weight=5, gamma=0.1, n_jobs=-1)
model_lgb = lgb.LGBMRegressor(n_estimators=500, max_depth=9, min_child_weight=5, n_jobs=-1)

- 학습을 시키고 성능을 테스트합니다.
- i9-10980XE CPU를 사용했을 때, 아래의 CV 시간은 약 800초 정도 소요됩니다.

In [ ]:
import tensorflow as tf
for e in train_X['dong_embedded']:
    print(tf.reshape(e, [-1]).numpy())
    break

In [ ]:
train_X['dong_embedded'][0

In [ ]:
models = []
scores = []
train_data = lgb.Dataset(train_X, label=train_y)
for model in [model_lgb]:
    model_name, mean_score = print_rmse_score(model)
    models.append(model_name)
    scores.append(mean_score)

In [ ]:
result_df = pd.DataFrame({'Model': models, 'Score': scores}).reset_index(drop=True)
result_df

In [ ]:
f, ax = plt.subplots(figsize=(10, 6))
plt.xticks(rotation='90')
sns.barplot(x=result_df['Model'], y=result_df['Score'])
plt.xlabel('Models', fontsize=15)
plt.ylabel('Model Performance', fontsize=15)
plt.ylim(0.22, 0.32)
plt.title('RMSLE', fontsize=15)
plt.show()

- CV 결과 LightGBM의 평균 RMSLE가 0.2395로 가장 작은 것을 알 수 있습니다.
- 이제, LightGBM을 이용하여 하이퍼 파라미터 튜닝을 진행하겠습니다. 

### Hyperparameter Tuning
- 시간적 비용을 고려하여, 하이퍼 파라미터 튜닝에서는 TimeSeries CV를 사용하지 않겠습니다.
- 대신 train 데이터를 8:2 비율로 분할하여 학습과 검증에 사용하겠습니다.
- 마찬가지로 데이터가 섞이지 않게, 검증 데이터는 train 데이터의 마지막 20%을 사용하겠습니다.

In [ ]:
cut = int(len(train_df)*0.8)
h_train = train_df[:cut]
h_valid = train_df[cut:]

h_train_X = h_train.drop('log_price', axis=1)
h_train_y = h_train['log_price']
h_valid_X = h_valid.drop('log_price', axis=1)
h_valid_y = h_valid['log_price']
print(h_train_X.shape, h_train_y.shape, h_valid_X.shape, h_valid_y.shape)

In [ ]:
h_train_X.head()

- optuna를 사용하여 하이퍼 파라미터 튜닝을 진행합니다.
- 약 20분 정도 진행됩니다.

In [ ]:
from optuna.samplers import TPESampler

sampler = TPESampler(seed=10)

def objective(trial):
    dtrain = lgb.Dataset(h_train_X, label=h_train_y)
    dtest = lgb.Dataset(h_valid_X, label=h_valid_y)

    param = {
        'objective': 'regression', # 회귀
        'verbose': -1,
        'metric': 'rmse', 
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }

    model = lgb.LGBMRegressor(**param)
    lgb_model = model.fit(h_train_X, h_train_y, eval_set=[(h_valid_X, h_valid_y)], verbose=0, early_stopping_rounds=25)
    rmse = RMSE(h_valid_y, lgb_model.predict(h_valid_X))
    return rmse
        
study_lgb = optuna.create_study(direction='minimize', sampler=sampler)
study_lgb.optimize(objective, n_trials=100)

In [ ]:
trial = study_lgb.best_trial
trial_params = trial.params
print('Best Trial: score {},\nparams {}'.format(trial.value, trial_params))

- 처음에 LightGBM 평균 RMSLE가 0.2390임을 감안했을때, 매우 성능이 향상됐음을 알 수 있습니다.

## LightGBM Submission
- 이제 test 셋을 사용해서 inference를 해보고 실제 점수를 Dacon에서 확인해볼시간입니다.
- 먼저 train 데이터에서 진행한 모든 전처리를 test 데이터에도 적용합니다.

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
final_lgb_model = lgb.LGBMRegressor(**trial_params)
final_lgb_model.fit(train_X, train_y)
final_lgb_pred = final_lgb_model.predict(test_df)

In [ ]:
final_lgb_pred

In [ ]:
plt.barh(train_X.columns, final_lgb_model.feature_importances_)

- 동, 완공연도, 면적, 아파트명 순으로 중요도가 높다고 나옵니다.
- top 시공사는 별로 중요하지 않다고 하네요..

## Submission
- 이제 제출을 해봅시다.
- 가격을 다시 역변환합니다.

In [ ]:
final_pred_sub = np.expm1(final_lgb_pred)
final_pred_sub

In [ ]:
sub = pd.read_csv('./input/test.csv')
sub_df = pd.DataFrame({'transaction_id': sub['transaction_id'], 'transaction_real_price': final_pred_sub})
sub_df

In [ ]:
sub_df.to_csv('submission_lgb.csv', index=False)

여기까지 따라오시느라 고생하셨습니다.

긴 글 봐주셔서 감사합니다:)

In [ ]:
trial = study_lgb.best_trial
trial_params = trial.params
print('Best Trial: score {},\nparams {}'.format(trial.value, trial_params))

- 처음에 LightGBM 평균 RMSLE가 0.2390임을 감안했을때, 매우 성능이 향상됐음을 알 수 있습니다.

## LightGBM Submission
- 이제 test 셋을 사용해서 inference를 해보고 실제 점수를 Dacon에서 확인해볼시간입니다.
- 먼저 train 데이터에서 진행한 모든 전처리를 test 데이터에도 적용합니다.

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
final_lgb_model = lgb.LGBMRegressor(**trial_params)
final_lgb_model.fit(train_X, train_y)
final_lgb_pred = final_lgb_model.predict(test_df)

In [ ]:
final_lgb_pred

In [ ]:
plt.barh(train_X.columns, final_lgb_model.feature_importances_)

- 동, 완공연도, 면적, 아파트명 순으로 중요도가 높다고 나옵니다.
- top 시공사는 별로 중요하지 않다고 하네요..

## Submission
- 이제 제출을 해봅시다.
- 가격을 다시 역변환합니다.

In [ ]:
final_pred_sub = np.expm1(final_lgb_pred)
final_pred_sub

In [ ]:
sub = pd.read_csv('./input/test.csv')
sub_df = pd.DataFrame({'transaction_id': sub['transaction_id'], 'transaction_real_price': final_pred_sub})
sub_df

In [ ]:
sub_df.to_csv('submission_lgb.csv', index=False)

여기까지 따라오시느라 고생하셨습니다.

긴 글 봐주셔서 감사합니다:)